In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import causal_ccm as ccm

# Load data
with open('data_norm.pkl','rb') as f:
    cld,pre,pol=pickle.load(f)

In [9]:
mask1=np.isnan(cld).sum(axis=0)<16
mask2=np.isnan(pre).sum(axis=0)<16
mask3=np.isnan(pol).sum(axis=0)<16
mask=mask1 & mask2 & mask3
print('Masked variables: ',mask1.sum(),mask2.sum(),mask3.sum())
print('Masked samples: ',mask.sum())

Masked variables:  355078 483423 146092
Masked samples:  144994


In [14]:
import pandas as pd
def interpolate(x,y):
    S=[0,0,0]
    S[0]=pd.Series(cld[:,x,y])
    S[1]=pd.Series(pre[:,x,y])
    S[2]=pd.Series(pol[:,x,y])

    for i in range(3):
        if np.isnan(S[i][0]):
            S[i][0]=S[i].mean()
        if np.isnan(S[i][47]):
            S[i][47]=S[i].mean()
        S[i]=S[i].interpolate()

    return np.array(S[0]),np.array(S[1]),np.array(S[2])


In [15]:
interpolate(600,1000)

(array([0.38273861, 0.1739204 , 0.10256263, 0.42662487, 0.85820244,
        0.63560639, 0.61779414, 0.71610277, 0.50040495, 0.39977126,
        0.12506665, 0.38598722, 0.28716403, 0.18828278, 0.16029292,
        0.40049669, 0.50130773, 0.81001921, 0.69582605, 0.63317641,
        0.29582431, 0.23730839, 0.        , 0.51262974, 0.22959184,
        0.45843773, 0.97006284, 0.40879254, 0.44821124, 0.25874836,
        0.29470482, 0.56893073, 0.30646008, 0.16819823, 0.14484801,
        0.09045905, 0.46394328, 0.17532481, 0.60758468, 0.71729897,
        0.74850543, 0.39464842, 0.56178468, 1.        , 0.69203112,
        0.07809167, 0.04802182, 0.02931207]),
 array([6.8084799e-02, 6.4282708e-02, 1.1884006e-02, 1.0735798e-01,
        3.9262486e-01, 4.1652536e-01, 3.6430690e-01, 3.8059577e-01,
        6.9509429e-01, 6.0349840e-01, 1.5533158e-01, 5.9215654e-02,
        1.6857415e-01, 3.0392639e-02, 3.5353981e-02, 6.3457832e-02,
        3.1738257e-01, 8.6965472e-01, 3.0260020e-01, 7.0558947e-01,
  

In [21]:
L=48;tau=1;E=2
L_range=range(3,49)
def calculateCCM(X,Y,Z):
    ccm_X_Y=ccm.ccm(X,Y,tau,E,L)
    ccm_Y_X=ccm.ccm(Y,X,tau,E,L)
    ccm_X_Z=ccm.ccm(X,Z,tau,E,L)
    ccm_Z_X=ccm.ccm(Z,X,tau,E,L)
    ccm_Y_Z=ccm.ccm(Y,Z,tau,E,L)
    ccm_Z_Y=ccm.ccm(Z,Y,tau,E,L)
    [rXY,pXY]=ccm_X_Y.causality()
    [rYX,pYX]=ccm_Y_X.causality()
    [rXZ,pXZ]=ccm_X_Z.causality()
    [rZX,pZX]=ccm_Z_X.causality()
    [rYZ,pYZ]=ccm_Y_Z.causality()
    [rZY,pZY]=ccm_Z_Y.causality()
    return np.array([rXY,rYX,rXZ,rZX,rYZ,rZY]),np.array([pXY,pYX,pXZ,pZX,pYZ,pZY])

In [22]:
calculateCCM(*interpolate(600,1000))

(array([ 0.47842372,  0.33921758,  0.31963486, -0.28794673,  0.24986314,
         0.07178538]),
 array([0.0006702 , 0.01967083, 0.02851634, 0.04968149, 0.09029697,
        0.63158071]))

In [29]:
def getCCMresult(x,y):
    if mask[x,y]==False:
        return np.nan*np.ones(6)
    X,Y,Z=interpolate(x,y)
    r,p=calculateCCM(X,Y,Z)
    r_modified=np.where(p<0.05,r,np.nan)
    return r_modified

In [30]:
getCCMresult(0,0)

Processing 0,0


array([nan, nan, nan, nan, nan, nan])

In [32]:
from joblib import Parallel,delayed
results=Parallel(n_jobs=-1,verbose=10)(delayed(getCCMresult)(x,y) for x in range(1118) for y in range(1372))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1962899448299436s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.15682363510131836s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.15465068817138672s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   10.3s
[Parallel(n_jobs

In [33]:
with open('results.pkl','wb') as f:
    pickle.dump(results,f)